In [1]:
import cv2
import glob
import os
import numpy as np
import math
from PIL import Image

bilinear_dir = os.path.join(os.getcwd(), "bilinear")
pad_dir = os.path.join(os.getcwd(), "pad")
nopad_dir = os.path.join(os.getcwd(), "nopad")
truth_dir = os.path.join(os.getcwd(), "truth")

In [2]:
# read image from directory
img_bilinear = glob.glob(os.path.join(bilinear_dir, "*.jpg"))
img_pad = glob.glob(os.path.join(pad_dir, "*.jpg"))
img_nopad = glob.glob(os.path.join(nopad_dir, "*.jpg"))
img_truth = glob.glob(os.path.join(truth_dir, "*.jpg"))

In [3]:
# get edge
def get_edge(image, edge = 6):
    width, length, _ = image.shape
    for x in range(edge, width-edge):
        for y in range(edge, length-edge):
            image[x,y,] = [0,0,0]
    return(image)

def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [4]:
# img1 = cv2.imread(img_bilinear[0])
# get_edge(img1)
# i = Image.fromarray(img1)
# i.show()


In [5]:
# compare edge
edge = 6
psnr_bi_pd = []
# diff_bi_pd_sum = []

for bi, pd, ti in zip(img_bilinear, img_pad, img_truth):    
    bilinear_image = cv2.imread(bi)
    pad_image = cv2.imread(pd)
    truth_image = cv2.imread(ti)
    
    bilinear_edge = get_edge(bilinear_image)
    pad_edge = get_edge(pad_image)

    psnr1 = psnr(truth_image, bilinear_edge)
    psnr2 = psnr(truth_image, pad_edge)
    psnr_bi_pd.append(psnr1-psnr2)


diff_sum1 = sum(psnr_bi_pd)
if diff_sum1 < 0:
    print("SRCNN with padding is better for edge area")
else:
    print("Bilinear is better for edge area")
    
    

Bilinear is better for edge area


In [6]:
# compare center
edge = 6
psnr_npd_pd = []
# diff_np_pd_sum = []

for npd, pd in zip(img_nopad, img_pad):    
    nopad_image = cv2.imread(np)
    pad_image = cv2.imread(pd)
    
    pad_center = pad_image[6:-6, 6:-6, :]
    
    psnr3 = psnr(truth_image, nopad_image)
    psnr4 = psnr(truth_image, pad_center)
    psnr_npd_pd.append(psnr3-psnr4)
    
diff_sum2 = sum(psnr_npd_pd)
if diff_sum2 < 0:
    print("SRCNN with padding is better for center area")
else:
    print("SRCNN with no padding is better for center area")



SRCNN with no padding is better for center area


In [7]:
# img0 = cv2.imread('/Users/wanyiellenzheng/Documents/GitHub/Fall2018-Proj3-Sec2-grp7 copy/TensorFlow-SRCNN/pad/img_0058.jpg')
# img0 = img0[6:-6, 6:-6, :]
# Image.fromarray(img0).show()
# Image.fromarray(img0).save(os.path.join(os.getcwd(), 'no_pad', 'img_0058.jpg'))